In [182]:
import pandas as pd

In [183]:
data = pd.read_excel('../dataset/Quantathon_Data_2025.xlsx')

In [184]:
data = data.drop(index=data.index[:6])

In [185]:
# First dataset: columns A, B, and C
first_dataset = data.iloc[:, :3]

# Second dataset: columns E, F, and G
second_dataset = data.iloc[:, 4:7]


In [186]:
first_dataset.columns = ['Date', 'S&P 500', 'Bond Rate']
first_dataset

,Date,S&P 500,Bond Rate
6,2024-12-31 00:00:00,5881.63,4.208
7,2024-12-30 00:00:00,5906.94,4.182
8,2024-12-27 00:00:00,5970.84,4.178
9,2024-12-26 00:00:00,6037.59,4.215
10,2024-12-24 00:00:00,6040.04,4.2
...,...,...,...
4528,2007-01-12 00:00:00,1430.73,4.95
4529,2007-01-11 00:00:00,1423.82,4.945
4530,2007-01-10 00:00:00,1414.85,4.91
4531,2007-01-09 00:00:00,1412.11,4.91


In [187]:
second_dataset.columns = ['Date', 'PrDec', 'PrInc']
second_dataset

,Date,PrDec,PrInc
6,2024-12-31 00:00:00,0.082,0.10128
7,2024-12-24 00:00:00,0.0778,0.09324
8,2024-12-18 00:00:00,0.08909,0.14135
9,2024-12-11 00:00:00,0.07816,0.10151
10,2024-12-04 00:00:00,0.07441,0.09571
...,...,...,...
4528,NaN,NaN,NaN
4529,NaN,NaN,NaN
4530,NaN,NaN,NaN
4531,NaN,NaN,NaN


In [188]:
merged_dataset = pd.merge(first_dataset, second_dataset, on='Date', how='outer').dropna(how='all').sort_values(by='Date')
merged_dataset

,Date,S&P 500,Bond Rate,PrDec,PrInc
3791,2007-01-08 00:00:00,1412.84,4.9,NaN,NaN
3792,2007-01-09 00:00:00,1412.11,4.91,NaN,NaN
3793,2007-01-10 00:00:00,1414.85,4.91,NaN,NaN
3794,2007-01-11 00:00:00,1423.82,4.945,NaN,NaN
3795,2007-01-12 00:00:00,1430.73,4.95,0.061515,0.056906
...,...,...,...,...,...
8314,2024-12-24 00:00:00,6040.04,4.2,0.0778,0.09324
8315,2024-12-26 00:00:00,6037.59,4.215,NaN,NaN
8316,2024-12-27 00:00:00,5970.84,4.178,NaN,NaN
8317,2024-12-30 00:00:00,5906.94,4.182,NaN,NaN


In [189]:
merged_dataset.to_csv('../dataset/Quantathon_Data_2025.csv', index=False)